<a href="https://www.kaggle.com/code/sergioadauto/da-covid-brazil?scriptVersionId=114264485" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="margin:0;">
    <div style="background-color: lightgray; border: 1px solid Black;  width: 85%; padding: 30px; margin:auto; text-align: center; border-radius: 10px 0; font-weight: bold;">
        <h1 style="text-align:center">Análise Exploratório dos Dados da Covid-19 no Brasil</h1>
    </div>
</div>

<h2>1. Contexto</h2>
<p style="text-align:justify"> Conforme o <a href="https://www.gov.br/saude/pt-br/coronavirus/o-que-e-o-coronavirus">Ministério da Saúde</a>, a Covid-19 é uma doença infecto respiratória causada pelo vírus da SARS-CoV-2, que apresenta elevado potencial de transmissibilidade de impacto global. Que afetou o mundo todo, e gerou mudanças de comportamento a respeito da precaução/proteção de doenças infectocontagiosas.</p>
<p style="text-align:justify">Por esse motivo, o estudo dos dados de transmissão e evolução dos casos em uma região, é fundamental para garantir o combate e controle da doença. Essa projeto visa construir um dashboard de visualização interativa, a respeito dos dados de vacinação e casos de Covid-19 no Brasil. Para esses dados serem consumidos, será utilizado o método de ETL (Extração - Transformação - Carregamento), e depois visualizados na plataforma Google Data Studio.</p>
<p style="text-align:justify">Para fazer isso, serão utilizadas duas bases de dados, a primeira vai conter informações sobre os casos da COVID-19, e a segunda informações de registro das vacinações. A primeira base (Casos de Covid-19), trata-se de um compilado de informações coletadas diariamente desde janeiro de 2020 pelo centro de ciência e engenharia da universidade <a href="https://www.jhu.edu/">John Hopkins</a>, que apresenta descrições geográficas de regiões de países (como estados, condados, províncias, etc.), além disso, seus dados podem ser consultados em seu <a href="https://systems.jhu.edu/research/public-health/ncov/">portal</a>, ou <a href="https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports">repositório</a> público. A segunda base de dados (Registro de Vacinação), refere-se a um compilado desenvolvido pelo projeto Nosso Mundo em Dados da universidade de <a href="https://www.ox.ac.uk/">Oxford</a>, que iniciou em janeiro de 2020, sendo atualizado diariamente com características geográficas de países, os dados podem ser acessados pelo <a href="https://ourworldindata.org">portal - Our World in Data</a>, ou através desse <a href="https://covid.ourworldindata.org/data/owid-covid-data.csv">link</a>.</p>

<h3>Descrição dos dados:</h3>
<p><strong>Caso de COVID-19:</strong></p>
    <ul>
         <li><em>date</em> = data de referência;
         <li><em>state</em> = estado;
         <li><em>country</em>: país; 
         <li><em>population</em> = população estimada;
         <li><em>confirmed</em> = número acumulado de infectados;
         <li><em>confirmed_1d</em> = número diário de infectados;
         <li><em>confirmed_moving_avg_7d</em> = média móvel de 7 dias do número diário de infectados;
         <li><em>confirmed_moving_avg_7d_rate_14d</em> = média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
         <li><em>deaths</em> = número acumulado de mortos;
         <li><em>deaths_1d</em> = número diário de mortos;
         <li><em>deaths_moving_avg_7d</em> = média móvel de 7 dias do número diário de mortos;
         <li><em>deaths_moving_avg_7d</em> = média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
         <li><em>month</em> = mês de referência;
         <li><em>year</em> = ano de referência.
    </ul>
<p><strong>Registros de Vacinação da COVID-19:</strong></p>
    <ul>
        <li><em>date</em> = data de referência;
        <li><em>country</em> = país;
        <li><em>population</em> = população estimada;
        <li><em>total</em> = número acumulado de doses administradas;
        <li><em>one_shot</em> = número acumulado de pessoas com uma dose;
        <li><em>one_shot_perc</em> = número acumulado relativo de pessoas com uma dose;
        <li><em>two_shots</em> = número acumulado de pessoas com duas doses;
        <li><em>two_shot_perc</em> = número acumulado relativo de pessoas com duas doses;
        <li><em>three_shots</em> = número acumulado de pessoas com três doses;
        <li><em>three_shot_perc</em> = número acumulado relativo de pessoas com três doses;
        <li><em>month</em> = mês de referência;
        <li><em>year</em> = ano de referência.
    </ul>


<h2>2. Pacotes e bibliotecas</h2>


In [1]:
import math
from typing import Iterator
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

<h2>3. Extração</h2>

In [2]:
#Carregando o DataFrame do Registro de vacinação:
df_vacinacao = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3], infer_datetime_format=True)


<p>Como os dados referente aos Casos de Covid-19 são registrados por dia em seu repositório, ou seja, existem vários arquivos com granularidade de cada dia. Será preciso fazer múltiplas extrações para montar um DataFrame com intervalo de tempo equivale ao DataFrame de Registro de Vacinação.</p>
<p>Primeiro, será preciso definir um intervalo de tempo. Para fazer isso, vou consultar a data mais recente do DataFrame de Vacinação referente ao Brasil, usando o método de consulta <code>.query()</code> do Pandas, e depois comparar o seu resultado com a data mais recente registrada no repositório de Casos de Covid (também referente ao Brasil), o qual é 05 de Dezembro de 2022:</p>

<details>
    <summary>print do repositório</summary>
<img src="https://github.com/SergioAdauto/da-covid-brazil/blob/main/image/print.png?raw=true" width="80%">
</details>

In [3]:
# Observando a data mais recente do DataFrame de vacinação:
display(str(df_vacinacao.query('location == "Brazil"')['date'].values[-1]))

'2022-12-18T00:00:00.000000000'

In [4]:
# Função responsável por definir um intervalo de tempo:
def intervalo_tempo(data_inicio: datetime, data_fim: datetime) -> Iterator[datetime]:
  intervalo_tempo_dias: int = (data_fim - data_inicio).days
  for data in range(intervalo_tempo_dias):
    yield data_inicio + timedelta(data)

In [5]:
# Definindo o intervalo de tempo:
data_inicio = datetime(2021,  1,  1)
data_fim = datetime(2022,  12,  5)

In [6]:
#Variável para registrar o Dataframe de casos de covid-19:
df_casos = None

casos_vazio = True # Variável responsável por validar se o dataframe está concluido.

#Loop para extrair os dados dados do repositório e selecionar as colunas desejadas:
for data in intervalo_tempo(data_inicio, data_fim):
    # Variável responsável por consultar a data desejada no repositório:  
    data_consulta = data.strftime('%m-%d-%Y')
    # link do repositório:
    url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{data_consulta}.csv'
    # Carregando os dados de casos do dia em um dataframe:
    caso = pd.read_csv(filepath_or_buffer=url, sep=',')
    # Apagando as colunas que não serão utilizadas:
    caso = caso.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
    # Selecionando as informações do Brasil:
    caso = caso.query('Country_Region == "Brazil"').reset_index(drop=True)
    # Convertendo a coluna 'Date' para o formato de data:
    caso['Date'] = pd.to_datetime(data.strftime('%Y-%m-%d'))
    #Verificando se o dataframe está completo: 
    if casos_vazio:
        df_casos = caso
        casos_vazio = False
    else:
        df_casos = df_casos.append(caso, ignore_index=True)

<h2>4. Transformação</h2>

<h4>4.1 Transformação dos dados de caso:</h4>

In [7]:
# Renomeando as colunas "Province_State" e "Country_Region" para
# facilitar sua compreensão:
df_casos = df_casos.rename(
  columns={
    'Province_State': 'state',
    'Country_Region': 'country'
  }
)

# Deixando colunas em minúscula:
for column in df_casos.columns:
  df_casos.rename(columns={column: column.lower()}, inplace=True)

In [8]:
# Verificando o nome dos estados:
df_casos.state.unique()

array(['Acre', 'Alagoas', 'Amapa', 'Amazonas', 'Bahia', 'Ceara',
       'Distrito Federal', 'Espirito Santo', 'Goias', 'Maranhao',
       'Mato Grosso', 'Mato Grosso do Sul', 'Minas Gerais', 'Para',
       'Paraiba', 'Parana', 'Pernambuco', 'Piaui', 'Rio Grande do Norte',
       'Rio Grande do Sul', 'Rio de Janeiro', 'Rondonia', 'Roraima',
       'Santa Catarina', 'Sao Paulo', 'Sergipe', 'Tocantins'],
      dtype=object)

<p><strong>OBS:</strong> Como alguns estados brasileiros estão com seu nome sem acentuação, será preciso ajustar isso para facilitar a geolocalização do mesmo.</p>

In [9]:
# Dicionário com nome dos estados que possuem acentuação:
estados = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}
# Ajustando nome de alguns estados brasileiros:
df_casos['state'] = df_casos['state'].apply(lambda state: estados.get(state) if state in estados.keys() else state)

<p>Enriquecimento da base de dados:</p>

In [10]:
# Criando coluna de Ano:
df_casos['year']  = df_casos['date'].apply(lambda date: date.strftime('%Y'))

# Criando coluna de Mês:
df_casos['month'] = df_casos['date'].apply(lambda date: date.strftime('%Y-%m'))

#Criando coluna da população estimada 
df_casos['population'] = round(100000 * (df_casos['confirmed'] / df_casos['incident_rate']))

#Eliminando a coluna de taxa de incidentes:
df_casos.drop('incident_rate', axis=1, inplace=True)

<p>Criando as colunas da média móvel de 7 dias, e estabilidade (14 dias) de casos e mortes por estado brasileiros:</p>


In [11]:
"""
#Variável para registrar os casos:
casos = None
# Para testar os casos vazio:
casos_vazio = True

#Função para verificar a tendência dos casos:
def tendencia(taxa: float) -> str:
    if np.isnan(taxa):
        return np.NaN
    if taxa < 0.75:
        status = 'redução'
    elif taxa > 1.15:
        status = 'aumento'
    else:
        status = 'estabilidade'

    return status
 
    
# Loop para verificar os casos por estados brasileiro:
for state in df_casos['state'].unique():
    casos_por_estado = df_casos.query(f'state == "{state}"').reset_index(drop=True)
    casos_por_estado = casos_por_estado.sort_values(by=['date'])
    # Casos confirmados:
    casos_por_estado['confirmed_1d'] = casos_por_estado['confirmed'].diff(periods=1)
    casos_por_estado['confirmed_moving_avg_7d'] = np.ceil(casos_por_estado['confirmed_1d'].rolling(window=7).mean())
    casos_por_estado['confirmed_moving_avg_7d_rate_14d'] = casos_por_estado['confirmed_moving_avg_7d']/casos_por_estado['confirmed_moving_avg_7d'].shift(periods=14)
    casos_por_estado['confirmed_trend'] = casos_por_estado['confirmed_moving_avg_7d_rate_14d'].apply(tendencia)
    # Casos que registraram mortes:
    casos_por_estado['deaths_1d'] = casos_por_estado['deaths'].diff(periods=1)
    casos_por_estado['deaths_moving_avg_7d'] = np.ceil(casos_por_estado['deaths_1d'].rolling(window=7).mean())
    casos_por_estado['deaths_moving_avg_7d_rate_14d'] = casos_por_estado['deaths_moving_avg_7d']/casos_por_estado['deaths_moving_avg_7d'].shift(periods=14)
    casos_por_estado['deaths_trend'] = casos_por_estado['deaths_moving_avg_7d_rate_14d'].apply(tendencia)
    # Verificando se os casos por estado estão registrado como vazio:
    if casos_vazio:
        casos = casos_por_estado
        casos_vazio = False
    else:
        casos = casos.append(casos_por_estado, ignore_index=True)

    # Salva as informações no DataFrame:
    df_casos = casos
    casos = None
"""
cases_ = None
cases_is_empty = True

def get_trend(rate: float) -> str:
    if np.isnan(rate):
        return np.NaN

    if rate < 0.75:
        status = 'downward'
    elif rate > 1.15:
        status = 'upward'
    else:
        status = 'stable'

    return status
  

for state in df_casos['state'].drop_duplicates():
    cases_per_state = df_casos.query(f'state == "{state}"').reset_index(drop=True)
    cases_per_state = cases_per_state.sort_values(by=['date'])

    cases_per_state['confirmed_1d'] = cases_per_state['confirmed'].diff(periods=1)
    cases_per_state['confirmed_moving_avg_7d'] = np.ceil(cases_per_state['confirmed_1d'].rolling(window=7).mean())
    cases_per_state['confirmed_moving_avg_7d_rate_14d'] = cases_per_state['confirmed_moving_avg_7d']/cases_per_state['confirmed_moving_avg_7d'].shift(periods=14)
    cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7d_rate_14d'].apply(get_trend)

    cases_per_state['deaths_1d'] = cases_per_state['deaths'].diff(periods=1)
    cases_per_state['deaths_moving_avg_7d'] = np.ceil(cases_per_state['deaths_1d'].rolling(window=7).mean())
    cases_per_state['deaths_moving_avg_7d_rate_14d'] = cases_per_state['deaths_moving_avg_7d']/cases_per_state['deaths_moving_avg_7d'].shift(periods=14)
    cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7d_rate_14d'].apply(get_trend)

    if cases_is_empty:
        cases_ = cases_per_state
        cases_is_empty = False
    else:
        cases_ = cases_.append(cases_per_state, ignore_index=True)

df_casos = cases_
cases_ = None

In [12]:
# Verificando os tipos das colunas:
df_casos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18981 entries, 0 to 18980
Data columns (total 16 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   state                             18981 non-null  object        
 1   country                           18981 non-null  object        
 2   confirmed                         18981 non-null  int64         
 3   deaths                            18981 non-null  int64         
 4   date                              18981 non-null  datetime64[ns]
 5   year                              18981 non-null  object        
 6   month                             18981 non-null  object        
 7   population                        18981 non-null  float64       
 8   confirmed_1d                      18954 non-null  float64       
 9   confirmed_moving_avg_7d           18792 non-null  float64       
 10  confirmed_moving_avg_7d_rate_14d  18414 non-nu

In [13]:
# Ajustando os tipos das colunas criadas:
df_casos = df_casos.astype({'population':np.int64, 'confirmed_1d':np.int64, 'confirmed_moving_avg_7d':np.int64, 'deaths_1d':np.int64,'deaths_moving_avg_7d':np.int64}, errors='ignore')

# Reorganizando as colunas:
df_casos = df_casos[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_1d', 'confirmed_moving_avg_7d', 'confirmed_moving_avg_7d_rate_14d', 'confirmed_trend', 'deaths', 'deaths_1d', 'deaths_moving_avg_7d', 'deaths_moving_avg_7d_rate_14d', 'deaths_trend', 'month', 'year']]

<h4>4.2 Transformação dos dados de vacinação:</h4>


In [14]:
# Selecionando os elementos do Brasil:
df_vacinacao = df_vacinacao.query('location == "Brazil" and date >= "2021-01-01" and date <= "2021-12-05"').reset_index(drop=True)

# Selecionado as colunas do dataframe de vacinação:
df_vacinacao = df_vacinacao[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

# Verificando se existe elementos faltando:
df_vacinacao.isna().sum()

location                     0
population                   0
total_vaccinations          16
people_vaccinated           16
people_fully_vaccinated     35
total_boosters             244
date                         0
dtype: int64

<p>Tratando os elementos faltantes do DataFrame:</p>

In [15]:
# Replicando os elementos faltantes com as informações da linha anterior:
df_vacinacao = df_vacinacao.fillna(method='ffill')

# Reorganizando o DataFrame de Vacinação:
df_vacinacao.rename(columns={'location': 'country',
                                            'total_vaccinations': 
                                            'total','people_vaccinated': 'one_shot',
                                            'people_fully_vaccinated': 'two_shots',
                                            'total_boosters': 'three_shots'}, inplace=True)


<p>Agora vou enriquecer o dataframe de vacinação:</p>

In [16]:
# Criando uma coluna ano:
df_vacinacao['year']  = df_vacinacao['date'].apply(lambda date: date.strftime('%Y'))

#Criando coluna de mês:
df_vacinacao['month'] = df_vacinacao['date'].apply(lambda date: date.strftime('%Y-%m'))

#Montando as colunas de proporções de dose de vacinação (1ª, 2ª e 3ª dose):
df_vacinacao['one_shot_perc'] = round(df_vacinacao['one_shot'] / df_vacinacao['population'], 4)
df_vacinacao['two_shots_perc'] = round(df_vacinacao['two_shots'] / df_vacinacao['population'], 4)
df_vacinacao['three_shots_perc'] = round(df_vacinacao['three_shots'] / df_vacinacao['population'], 4)

#verificando o tipos dos dados:
df_vacinacao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339 entries, 0 to 338
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   country           339 non-null    object        
 1   population        339 non-null    float64       
 2   total             323 non-null    float64       
 3   one_shot          323 non-null    float64       
 4   two_shots         304 non-null    float64       
 5   three_shots       95 non-null     float64       
 6   date              339 non-null    datetime64[ns]
 7   year              339 non-null    object        
 8   month             339 non-null    object        
 9   one_shot_perc     323 non-null    float64       
 10  two_shots_perc    304 non-null    float64       
 11  three_shots_perc  95 non-null     float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 31.9+ KB


In [17]:
# Ajustando os tipos das colunas:
df_vacinacao.astype({'population':np.int64, 'total':np.int64, 'one_shot':np.int64, 'two_shots':np.int64, 'three_shots':np.int64}, errors='ignore')

#Organizando o DataFrame:
df_vacinacao = df_vacinacao[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'month', 'year']]

<h2>5. Carregamento</h2>
<p>Após manipular todos os dados de casos de covid e vacinação, agora será persistido em disco os dois DataFrames, para assim serem utilizados no Google Data Studio.</p>

In [18]:
df_casos.to_csv('./covid-cases.csv', sep=',', index=False)
df_vacinacao.to_csv('./covid-vaccines.csv', sep=',', index=False)